In [ ]:
# 모듈 import
import multiprocessing
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.utils import get_device
from mmdet.apis import set_random_seed

In [ ]:
# config file 들고오기
config = './configs/_trash_/universenet101_trash_detection.py'
cfg = Config.fromfile(config)

In [ ]:
cfg.seed = 2022
cfg.deterministic = True
set_random_seed(2022, True)

cfg.data.samples_per_gpu = 8
cfg.data.workers_per_gpu = multiprocessing.cpu_count() // 2 # num_workers

cfg.gpu_ids = [0]
cfg.work_dir = './work_dir/universenet101_trash_detection.py'
cfg.device = get_device()

In [ ]:
#wandb
import datetime

now = (datetime.datetime.now().replace(microsecond=0) + datetime.timedelta(hours=9)).strftime("%m-%d %H:%M")
cfg.log_config.hooks = [
    dict(type='TextLoggerHook'),
    dict(type='WandbLoggerHook',
         init_kwargs={
            'project': 'trash_detection',
            'entity': 'kimhs21474',
            'name' : f'{cfg.model.type}_{now}_pkt',
            'tags': [cfg.model.backbone.type, cfg.model.neck[0].type, 'multiscale']
            },
         interval=10,
        )]

In [ ]:
# build_dataset
datasets = [build_dataset(cfg.data.train)]
datasets[0]

In [ ]:
# 모델 build 및 pretrained network 불러오기
model = build_detector(cfg.model)
model.init_weights()

In [ ]:
meta = dict()
meta['seed'] = cfg.seed

In [ ]:
# 모델 학습
train_detector(model, datasets[0], cfg, distributed=False, validate=True, meta=meta)